# Generating manifests

## Pods

In [3]:
kubectl run nginx --generator run-pod/v1 --image ngnix --dry-run -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    run: nginx
  name: nginx
spec:
  containers:
  - image: ngnix
    name: nginx
    resources: {}
  dnsPolicy: ClusterFirst
  restartPolicy: Always
status: {}


---

## Deployments


In [5]:
kubectl run  nginx --image nginx --dry-run -o yaml 

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
apiVersion: apps/v1
kind: Deployment
metadata:
  creationTimestamp: null
  labels:
    run: nginx
  name: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      run: nginx
  strategy: {}
  template:
    metadata:
      creationTimestamp: null
      labels:
        run: nginx
    spec:
      containers:
      - image: nginx
        name: nginx
        resources: {}
status: {}


In [8]:
kubectl run nginx --image nginx --replicas 4 --dry-run -o yaml

kubectl run --generator=deployment/apps.v1 is DEPRECATED and will be removed in a future version. Use kubectl run --generator=run-pod/v1 or kubectl create instead.
apiVersion: apps/v1
kind: Deployment
metadata:
  creationTimestamp: null
  labels:
    run: nginx
  name: nginx
spec:
  replicas: 4
  selector:
    matchLabels:
      run: nginx
  strategy: {}
  template:
    metadata:
      creationTimestamp: null
      labels:
        run: nginx
    spec:
      containers:
      - image: nginx
        name: nginx
        resources: {}
status: {}


---

## Services

In [9]:
kubectl expose -h

Expose a resource as a new Kubernetes service.

 Looks up a deployment, service, replica set, replication controller or pod by
name and uses the selector for that resource as the selector for a new service
on the specified port. A deployment or replica set will be exposed as a service
only if its selector is convertible to a selector that service supports, i.e.
when the selector contains only the matchLabels component. Note that if no port
is specified via --port and the exposed resource has multiple ports, all will be
re-used by the new service. Also if no labels are specified, the new service
will re-use the labels from the resource it exposes.

 Possible resources include (case insensitive):

 pod (po), service (svc), replicationcontroller (rc), deployment (deploy),
replicaset (rs)

Examples:
  # Create a service for a replicated nginx, which serves on port 80 and
connects to the containers on port 8000.
  kubectl expose rc nginx --port=80 --target-port=8000
  
  # Create a service 

### Create a Service named redis-service of type ClusterIP to expose pod redis on port 6379


In [14]:
kubectl expose pod redis --port 6379 --name redis-service --dry-run -o yaml
# (This will automatically use the pod's labels as selectors)

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: null
  labels:
    run: redis
  name: redis-service
spec:
  ports:
  - port: 6379
    protocol: TCP
    targetPort: 6379
  selector:
    run: redis
status:
  loadBalancer: {}


In [16]:
# or
kubectl create service clusterip redis --tcp=6379:6379 --dry-run -o yaml
# (This will not use the pods labels as selectors, instead it will assume selectors as app=redis.

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: null
  labels:
    app: redis
  name: redis
spec:
  ports:
  - name: 6379-6379
    port: 6379
    protocol: TCP
    targetPort: 6379
  selector:
    app: redis
  type: ClusterIP
status:
  loadBalancer: {}


### Create a Service named nginx of type NodePort to expose pod nginx's port 80 on port 30080 on the nodes:

In [20]:
kubectl expose pod nginx --type NodePort --port 80 --name nginx-service --dry-run -o yaml
# This will automatically use the pod's labels as selectors,
# but you cannot specify the node port. You have to generate
# a definition file and then add the node port in manually
# before creating the service with the pod.)

Error from server (NotFound): pods "nginx" not found


: 1

In [23]:
# or
kubectl create service nodeport nginx --tcp=80:80 --node-port=30080 --dry-run -o yaml
# (This will not use the pods labels as selectors)

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: null
  labels:
    app: nginx
  name: nginx
spec:
  ports:
  - name: 80-80
    nodePort: 30080
    port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: nginx
  type: NodePort
status:
  loadBalancer: {}


Both the above commands have their own challenges. While one of it cannot accept a selector the other cannot accept a node port. I would recommend going with the `kubectl expose` command. If you need to specify a node port, generate a definition file using the same command and manually input the nodeport before creating the service.

